# The Simplex algotithm

A simple implementation for the primal formulation of the simplex algorithm.

## Formulation of a simple problem

In [1]:
import numpy as np
import attr
from fractions import Fraction

In [2]:
_A = np.array(
    [
        [1, 3, 1, 1, 0, 0, 0],
        [-1, 0, 3, 0, 1, 0, 0],
        [2, -1, 2, 0, 0, 1, 0],
        [2, 3, -1, 0, 0, 0, 1]
    ],
    dtype=np.int64
)

_A

array([[ 1,  3,  1,  1,  0,  0,  0],
       [-1,  0,  3,  0,  1,  0,  0],
       [ 2, -1,  2,  0,  0,  1,  0],
       [ 2,  3, -1,  0,  0,  0,  1]])

In [3]:
_p = np.array([3, 2, 4, 2], dtype=np.int64)
_p

array([3, 2, 4, 2])

In [4]:
_B = np.hstack([_A, _p[..., np.newaxis]])
_B

array([[ 1,  3,  1,  1,  0,  0,  0,  3],
       [-1,  0,  3,  0,  1,  0,  0,  2],
       [ 2, -1,  2,  0,  0,  1,  0,  4],
       [ 2,  3, -1,  0,  0,  0,  1,  2]])

In [5]:
_z = np.array([5, 5, 3, 0, 0, 0, 0] + [0], dtype=np.int64)
_z

array([5, 5, 3, 0, 0, 0, 0, 0])

In [6]:
from mcf_simplex_analyzer.fractionarray import FractionArray

In [7]:
A = FractionArray(_A, np.ones_like(_A))
A

FractionArray(numerator=[[ 1  3  1  1  0  0  0]
 [-1  0  3  0  1  0  0]
 [ 2 -1  2  0  0  1  0]
 [ 2  3 -1  0  0  0  1]], denominator=[[1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]])

In [8]:
A[..., 1]

FractionArray(numerator=[ 3  0 -1  3], denominator=[1 1 1 1])

In [9]:
p = FractionArray(_p, np.ones_like(_p))
p

FractionArray(numerator=[3 2 4 2], denominator=[1 1 1 1])

In [10]:
B = FractionArray(_B, np.ones_like(_B))
B

FractionArray(numerator=[[ 1  3  1  1  0  0  0  3]
 [-1  0  3  0  1  0  0  2]
 [ 2 -1  2  0  0  1  0  4]
 [ 2  3 -1  0  0  0  1  2]], denominator=[[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]])

In [11]:
base_ind = np.array([3, 4, 5, 6, -1])
base = np.zeros_like(_B[0], dtype=bool)
base[base_ind] = True
base

array([False, False, False,  True,  True,  True,  True,  True])

In [12]:
# TODO: How to determine this?
row_to_var_index = np.array([3, 4, 5, 6])
var_index_to_row = np.array([0, 0, 0, 0, 1, 2, 3])

In [13]:
z = FractionArray(_z, np.ones_like(_z))
z

FractionArray(numerator=[5 5 3 0 0 0 0 0], denominator=[1 1 1 1 1 1 1 1])

## Entering variable (Dantzig)

In [14]:
def dantzig_entering(z, base):
    positive = np.where(~base)[0]
    entering = positive[z[positive].argmax()]
    
    return entering

entering = dantzig_entering(z, base)
entering

0

In [15]:
# Possible speedup: Convert to floats
rng = np.random.default_rng()
x = rng.uniform(size=1000)
m = x.max()
eps = 0.001
np.where((m - eps <= x) & (x <= m + eps))[0]

array([514])

## Leaving variable (Dantzig)

In [16]:
def dantzig_leaving(T, entering):   
    positive = np.where(T[..., entering].numerator >= 0)[0]
    
    bounds = T[..., -1][positive] / T[..., entering][positive]
    print("Bounds:", bounds)
    valid = np.where(bounds.numerator > 0)[0]
    leaving_row = positive[valid[bounds[valid].argmin()]]
    
    return leaving_row

In [17]:
leaving_row = dantzig_leaving(B, entering)
leaving_row

Bounds: FractionArray(numerator=[3 2 1], denominator=[1 1 1])


3

In [18]:
leaving = row_to_var_index[leaving_row]
leaving

6

## Update

In [19]:
def determine_update_row(B, entering, leaving_row):
    if B[leaving_row, entering] == 0:
        return FractionArray.from_array(np.ones_like(B[leaving_row].numerator))
    

    return B[leaving_row] / B[leaving_row, entering]

In [20]:
update_row = determine_update_row(B, entering, leaving_row)
update_row

FractionArray(numerator=[ 1  3 -1  0  0  0  1  1], denominator=[1 2 2 1 1 1 2 1])

In [21]:
B

FractionArray(numerator=[[ 1  3  1  1  0  0  0  3]
 [-1  0  3  0  1  0  0  2]
 [ 2 -1  2  0  0  1  0  4]
 [ 2  3 -1  0  0  0  1  2]], denominator=[[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]])

In [22]:
def update_table(B, update_row, entering, leaving_row):
    B[leaving_row] = update_row
    
    for row in range(B.numerator.shape[0]):
        if row == leaving_row:
            continue
            
        B[row] -= B[row, entering] * update_row

In [23]:
update_table(B, update_row, entering, leaving_row)
B

FractionArray(numerator=[[ 0  3  3  1  0  0 -1  2]
 [ 0  3  5  0  1  0  1  3]
 [ 0 -4  3  0  0  1 -1  2]
 [ 1  3 -1  0  0  0  1  1]], denominator=[[1 2 2 1 1 1 2 1]
 [1 2 2 1 1 1 2 1]
 [1 1 1 1 1 1 1 1]
 [1 2 2 1 1 1 2 1]])

In [24]:
def update_objective(z, update_row, entering, leaving_row):
    update = update_row * z[entering]
    update[:-1] *= -1
    
    z += update

In [25]:
update_objective(z, update_row, entering, leaving_row)
z

FractionArray(numerator=[ 0 -5 11  0  0  0 -5  5], denominator=[1 2 2 1 1 1 2 1])

In [26]:
def update_base(base, entering, leaving_row):
    leaving = row_to_var_index[leaving_row]
    row_to_var_index[leaving_row] = entering
    
    var_index_to_row[entering] = var_index_to_row[leaving]

    base[entering] = True
    base[leaving] = False
    
    return base
    
update_base(base, entering, leaving_row)
base

array([ True, False, False,  True,  True,  True, False,  True])

# Second iteration

In [27]:
entering2 = dantzig_entering(z, base)
entering2

2

In [28]:
leaving_row2 = dantzig_leaving(B, entering2)

Bounds: FractionArray(numerator=[4 6 2], denominator=[3 5 3])


In [29]:
leaving2 = row_to_var_index[leaving_row2]
leaving2

5

In [30]:
update_row = determine_update_row(B, entering2, leaving_row2)
update_row

FractionArray(numerator=[ 0 -4  1  0  0  1 -1  2], denominator=[1 3 1 1 1 3 3 3])

In [31]:
update_table(B, update_row, entering2, leaving_row2)
B

FractionArray(numerator=[[ 0  7  0  1  0 -1  0  1]
 [ 0 29  0  0  1 -5  4  4]
 [ 0 -4  1  0  0  1 -1  2]
 [ 1  5  0  0  0  1  1  4]], denominator=[[1 2 1 1 1 2 1 1]
 [1 6 1 1 1 6 3 3]
 [1 3 1 1 1 3 3 3]
 [1 6 1 1 1 6 3 3]])

In [32]:
type(Fraction(3, 2))

fractions.Fraction

In [33]:
update_objective(z, update_row, entering2, leaving_row2)
z

FractionArray(numerator=[  0  29   0   0   0 -11  -2  26], denominator=[1 6 1 1 1 6 3 3])

In [34]:
update_base(base, entering2, leaving_row2)
base

array([ True, False,  True,  True,  True, False, False,  True])

In [35]:
print("Objective function:", z[-1])
for var in row_to_var_index:
    print(f"x_{var}:", B[var_index_to_row[var], -1])

Objective function: 26/3
x_3: 1
x_4: 4/3
x_2: 2/3
x_0: 4/3


## Third iteration

In [36]:
entering2 = dantzig_entering(z, base)

leaving_row2 = dantzig_leaving(B, entering2)

leaving2 = row_to_var_index[leaving_row2]

update_row = determine_update_row(B, entering2, leaving_row2)

update_table(B, update_row, entering2, leaving_row2)
update_objective(z, update_row, entering2, leaving_row2)
update_base(base, entering2, leaving_row2)

Bounds: FractionArray(numerator=[2 8 8], denominator=[ 7 29  5])


array([ True,  True,  True,  True, False, False, False,  True])

In [37]:
print("Objective function:", z[-1])
for var in row_to_var_index:
    print(f"x_{var}:", B[var_index_to_row[var], -1])

Objective function: 10
x_3: 1/29
x_1: 8/29
x_2: 30/29
x_0: 32/29


## Simplex class

In [38]:
_C = np.array(
    [
        [1, 3, 1 ],
        [-1, 0, 3 ],
        [2, -1, 2 ],
        [2, 3, -1 ]
    ],
    dtype=np.int64
)

C = FractionArray.from_array(_C)
C

FractionArray(numerator=[[ 1  3  1]
 [-1  0  3]
 [ 2 -1  2]
 [ 2  3 -1]], denominator=[[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]])

In [39]:
_p = np.array([3, 2, 4, 2], dtype=np.int64)
_p
p = FractionArray.from_array(_p)
p

FractionArray(numerator=[3 2 4 2], denominator=[1 1 1 1])

In [40]:
_z = np.array([5, 5, 3], dtype=np.int64)
z = FractionArray.from_array(_z)
z

FractionArray(numerator=[5 5 3], denominator=[1 1 1])

In [41]:
from mcf_simplex_analyzer.simplex import (Simplex, LPFormType, LPFormulation)

In [42]:
form = LPFormulation(LPFormType.Canonical, C, p, z)
form

LPFormulation(type=<LPFormType.Canonical: 0>, table=FractionArray(numerator=[[ 1  3  1]
 [-1  0  3]
 [ 2 -1  2]
 [ 2  3 -1]], denominator=[[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]]), rhs=FractionArray(numerator=[3 2 4 2], denominator=[1 1 1 1]), objective=FractionArray(numerator=[5 5 3], denominator=[1 1 1]), meta={})

In [43]:
s = Simplex.instantiate(form)
s

Simplex(table=FractionArray(numerator=[[ 1  3  1  1  0  0  0  3]
 [-1  0  3  0  1  0  0  2]
 [ 2 -1  2  0  0  1  0  4]
 [ 2  3 -1  0  0  0  1  2]], denominator=[[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]]), _objective_row=FractionArray(numerator=[5 5 3 0 0 0 0 0], denominator=[1 1 1 1 1 1 1 1]), base=array([False, False, False,  True,  True,  True,  True]), _row_to_var_index=array([3, 4, 5, 6]), _var_index_to_row=array([0, 0, 0, 0, 1, 2, 3]))

In [44]:
s.table[..., :-1]

FractionArray(numerator=[[ 1  3  1  1  0  0  0]
 [-1  0  3  0  1  0  0]
 [ 2 -1  2  0  0  1  0]
 [ 2  3 -1  0  0  0  1]], denominator=[[1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]])

In [45]:
s.solve()

{'result': 'success',
 'value': Fraction(10, 1),
 'variables': FractionArray(numerator=[32  8 30  1  0  0  0  0], denominator=[29 29 29 29  1  1  1  1])}

In [46]:
s.state()

{'value': Fraction(10, 1),
 'variables': FractionArray(numerator=[32  8 30  1  0  0  0  0], denominator=[29 29 29 29  1  1  1  1])}

In [47]:
problem2 = LPFormulation(LPFormType.Canonical,
              table=FractionArray.from_array([[2,3,1], [4,1,2], [3, 4, 2]]),
              rhs=FractionArray.from_array([5, 11, 8]),
              objective=FractionArray.from_array([5,4,3])
             )
problem2

LPFormulation(type=<LPFormType.Canonical: 0>, table=FractionArray(numerator=[[2 3 1]
 [4 1 2]
 [3 4 2]], denominator=[[1 1 1]
 [1 1 1]
 [1 1 1]]), rhs=FractionArray(numerator=[ 5 11  8], denominator=[1 1 1]), objective=FractionArray(numerator=[5 4 3], denominator=[1 1 1]), meta={})

In [48]:
Simplex.instantiate(problem2).solve()

{'result': 'success',
 'value': Fraction(13, 1),
 'variables': FractionArray(numerator=[2 0 1 0 1 0 0], denominator=[1 1 1 1 1 1 1])}

## Degeneracy

In [49]:
problem3 = LPFormulation(LPFormType.Canonical,
              table=FractionArray.from_array([[0, 0, 2], [2, -4, 6], [-1, 3, 4]]),
              rhs=FractionArray.from_array([1, 3, 2]),
              objective=FractionArray.from_array([2, -1, 8])
             )
problem3

LPFormulation(type=<LPFormType.Canonical: 0>, table=FractionArray(numerator=[[ 0  0  2]
 [ 2 -4  6]
 [-1  3  4]], denominator=[[1 1 1]
 [1 1 1]
 [1 1 1]]), rhs=FractionArray(numerator=[1 3 2], denominator=[1 1 1]), objective=FractionArray(numerator=[ 2 -1  8], denominator=[1 1 1]), meta={})

In [50]:
Simplex.instantiate(problem3).solve()

{'result': 'success',
 'value': Fraction(27, 2),
 'variables': FractionArray(numerator=[17  7  0  1  0  0  0], denominator=[2 2 1 1 1 1 1])}

## Cycling

In [51]:
problem4 = LPFormulation(
    LPFormType.Canonical,
    table=FractionArray.from_array([
        [Fraction(1, 2), Fraction(-55, 10), -Fraction(25, 10), 9],
        [Fraction(1, 2), -Fraction(3,2), -Fraction(1, 2), 1],
        [1, 0, 0, 0]]
    ),
    rhs=FractionArray.from_array([0, 0, 1]),
    objective=FractionArray.from_array([10, -57, -9, -24])
)
problem4

LPFormulation(type=<LPFormType.Canonical: 0>, table=FractionArray(numerator=[[  1 -11  -5   9]
 [  1  -3  -1   1]
 [  1   0   0   0]], denominator=[[2 2 2 1]
 [2 2 2 1]
 [1 1 1 1]]), rhs=FractionArray(numerator=[0 0 1], denominator=[1 1 1]), objective=FractionArray(numerator=[ 10 -57  -9 -24], denominator=[1 1 1 1]), meta={})

In [52]:
Simplex.instantiate(problem4).solve(degenerate_max_iter=100)

{'result': 'cycle'}